 Facebook Coment Volume Prediction using Naive Bayes

In [1]:
#importamos las bibliotecas necesarias para el proyecto
import csv
import random
import math

Lo primero que tenemos que hacer es cargar el dataset que esta en un archivo csv

In [15]:
#funcion para cargar un archivo csv por lineas
def cargaCsv(file):
    lineas = csv.reader(open(file, "rt", encoding=<theencodingofthefile>))
    dataset = list(lineas)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

Vamos a dividir los datos contenidos en el csv en datos de train (para entenar) y datos de test (para probar). Esto lo hacemos con un radio definido por nosotros, en este proyecto usaremos 0.7 valor al que llegamos a travez de experimentación

In [16]:
#funcion para separar los datos en datos de entrenamiento y de test
def dividirDatos(dataset, radio):
    trainSize = int(len(dataset) * radio)
    c = list(dataset)
    trainSet = []
    while len(trainSet) < trainSize:
        index = random.randrange(len(c))
        trainSet.append(c.pop(index))
    return [trainSet, c]

Ahora separaremos los datos de entrenamiento por Clases

In [17]:
#funcion que separa los datos de entrenamiento por clases
def separarClass(dataset):
    otros = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in otros):
            otros[vector[-1]] = []
        otros[vector[-1]].append(vector)
    return otros

Será necesario también calcular la desviación estandar para esto necesitamos calcular la media de cada uno de los atributos 

In [18]:
#funcion que calcula la media
def media(numb):
    return sum(numb)/float(len(numb))

#funcion que calcula la desviacion estandar
def desvStdr(numb):
    promedio = media(numb)
    varianza = sum([pow(x-promedio,2) for x in numb])/float(len(numb))
    return math.sqrt(varianza)

Con esto ya podemos resumir los datos calculando la desviación estandar y la media de cada uno de los atributos

In [19]:
def resumir(dataset):
    res = [(media(atr), desvStdr(atr)) for atr in zip(*dataset)]
    del res[-1]
    return res

def resumenClass(dataset):
    res = {}
    otros = separarClass(dataset)
    for valor, instancia in otros.iteritems():
        res[valor] = resumir(instancia)
    return res

Con todo lo anterior ya podemos comenzar a hacer predicciones, lo primero que haremos será calcular la probabilidad Gaussiana

In [20]:
#funcion que calcula la probabilidad Gaussiana
def proba(x, media, des):
    expo = math.exp(-(math.pow(x-media,2)/(2*math.pow(des,2))))
    return (1 / (math.sqrt(2*math.pi) * des)) *	expo

Ahora combinamos las probabildades de cada uno de los atributos para calcular la probabilidad de la clase

In [21]:
#funcion para calcular la probabilidad de una clase
def classProba(res, vector):
    prob = {}
    for valor, res in res.iteritems():
        prob[valor] = 1
        for i in range(len(res)):
            media, des = res[i]
            x = vector[i]
            prob[valor] *= proba(x, media, des)
    return prob

Con los calculos de las probabilidades ya podemos comenzar a hacer predicciones

In [22]:
#funcion para hacer predicciones
def predecir(res, vector):
    prob = classProba(res, vector)
    etiqueta, mejor = None, -1
    for classValue, probability in prob.iteritems():
        if etiqueta is None or probability > mejor:
            mejor = probability
            etiqueta = classValue
    return etiqueta

#funcion para hacer predicciones para cada una de las instancias de los datos
def getPredicciones(res, conj):
    predicciones = []
    for i in range(len(conj)):
        ret = predecir(res, conj[i])
        predicciones.append(ret)
    return predicciones

Finalmente vamos a calcular la precisión de predicción del modelo

In [23]:
#funcion para calcular la precisión
def Accuracy(conj, predicciones):
    correct = 0
    for x in range(len(conj)):
        if conj[x][-1] == predicciones[x]:
            correct += 1
    return (correct/float(len(conj))) * 100.0

In [24]:
#funcion main
def main():
    dataset = cargaCsv('Dataset/Testing/Features_TestSet.csv')
    trainingSet, testSet = dividirDatos(dataset, 0.7)
    print('Cargados {0} train={1} test={2}').format(len(dataset), len(trainingSet), len(testSet))
    resu = resumenClass(trainingSet)
    print('Resumen: {0}%').format(resu)
    predicciones = getPredicciones(resu, testSet)
    print('predicciones: {0}').format(predicciones)
    acertadas = Accuracy(testSet, predicciones)
    print('Acertadas: {0}%').format(acertadas)

if __name__ == "__main__":
    main()

Cargados {0} train={1} test={2}


AttributeError: 'NoneType' object has no attribute 'format'